In [30]:
import urllib as u
from urllib.request import urlopen
import json
import pandas as pd
from pandas_datareader import data
import numpy as np
from tqdm import tqdm

api_key='64d94994afb9689c49b660ebe9dee39c'

In [2]:
# Function to read in json data from FMP
def get_json_data(url):
    '''
    Scrape data (which must be json format) from given url
    Input: url to financialmodelingprep API
    Output: json file
    '''
    response = urlopen(url)
    dat = response.read().decode('utf-8')
    return json.loads(dat)

In [3]:
# Function to get sector of a Stock from its profile json file
def find_sector(json):
    
    for k, v in a.items():
        if k == 'symbol':
            symbol=v

    for k, v in a.items():
        if k == 'profile':
            profile=v

    for k, v in profile.items():
        if k == 'sector':
            sector=v
    return(symbol, sector)

In [4]:
# Function to get Year 2019 Financial Indicators from json file
def find_fin_indicators(json, indicator):
    cat = ['financials','growth','metrics']
    
    for k, v in json.items():
        if k == 'symbol':
            symbol=v

    for k, v in json.items():
        if k in cat:
            category = v

    indicators_2019 = [[(k,v) for k,v in d.items()] for d in category if d['date'][:4] == '2019'] # Generator creates a list of lists
    indicators_2019_list = [item for sublist in indicators_2019 for item in sublist] # Creating a flat list

    a = []

    for tup in (x for x in indicators_2019_list if x[0] in indicator):
        a.append((symbol,) + tup)
    return a

In [5]:
# Function to get Year 2019 Ratios from json file
def find_ratios_indicators(json, indicator):
    global ratios, symbol
    
    for k in json.items():
        symbol = json['symbol']
    
    for k in json.items():
        ratios = json['ratios']
            
    ratios_2019 = [d for d in ratios if d['date'][:4] == '2019']
    
    unpack1 = ratios_2019[0]['investmentValuationRatios']
    unpack2 = ratios_2019[0]['profitabilityIndicatorRatios']
    unpack3 = ratios_2019[0]['operatingPerformanceRatios']
    unpack4 = ratios_2019[0]['liquidityMeasurementRatios']
    unpack5 = ratios_2019[0]['debtRatios']
    unpack6 = ratios_2019[0]['cashFlowIndicatorRatios']
    
    unpack = unpack1
    unpack.update(unpack2)
    unpack.update(unpack3)
    unpack.update(unpack4)
    unpack.update(unpack5)
    unpack.update(unpack6)
    
    full_set = []
    reduced_set =[]

    for row in unpack.items():
            full_set.append(row)
    
    for tup in (x for x in full_set if x[0] in indicator):
        reduced_set.append((symbol,) +tup)
    
    return reduced_set

In [81]:
# Function to get the stock's price variation during the year 2020
def get_price_var(symbol):

    prices = data.DataReader(symbol, 'yahoo', '2020-01-01', '2020-12-31')['Adj Close']

    end = prices.index[-1]
    start = prices.index[0]

    price_var = ((prices[end] / prices[start]) - 1) * 100
    return symbol, price_var

In [6]:
# Full list of available stocks from FMP
url = 'https://financialmodelingprep.com/api/v3/stock/list?apikey=' + api_key
ticks_json = get_json_data(url)

stocks = pd.DataFrame(ticks_json)
stocks = stocks[['symbol','name','exchange']]
stocks = stocks.dropna(how='any',subset=['symbol','name','exchange'])

In [7]:
# Limit the analysis to stocks of the New York Stock Exchange and Nasdaq Global
stocks_nyse = stocks[(stocks['exchange'] == 'Nasdaq Global Select') | (stocks['exchange'] == 'New York Stock Exchange')]

# For Testing purposes - To be dropped
stocks_nyse = stocks[(stocks['symbol'] == 'AAPL') | (stocks['symbol'] == 'WMT')]

stocks_nyse.reset_index(drop=True, inplace=True)
tickers_nyse = stocks_nyse['symbol'].tolist()

In [8]:
# Get sectors from company profiles
tickers_nyse_sector = []

for tick in tqdm(tickers_nyse):
    url = 'https://financialmodelingprep.com/api/v3/company/profile/' + tick + '?apikey=' + api_key 
    a = get_json_data(url)
    tickers_nyse_sector.append(find_sector(a))

100%|██████████| 2/2 [00:01<00:00,  1.53it/s]


In [9]:
tickers_nyse_sector = pd.DataFrame(tickers_nyse_sector)
tickers_nyse_sector.columns = ['symbol', 'sector']

stocks_nyse = stocks_nyse.merge(tickers_nyse_sector)

In [10]:
# Load financial indicators from the provided .txt file
indicators = []
filename = 'indicators.txt'
with open(filename, 'r') as f:
    for line in f:
        indicators.append(line.strip('\n'))

In [11]:
# Get all indicators
inc_statement_indicators = []
cash_flow_indicators = []
bs_indicators = []
fs_growth_indicators = []
metrics_indicators = []
ratios_indicators = []

for tick in tqdm(tickers_nyse):
    try:
        url0 = 'https://financialmodelingprep.com/api/v3/financials/income-statement/' + tick + '?apikey=' + api_key
        url1 = 'https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/' + tick + '?apikey=' + api_key
        url2 = 'https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/' + tick + '?apikey=' + api_key
        url3 = 'https://financialmodelingprep.com/api/v3/financial-statement-growth/' + tick + '?apikey=' + api_key
        url4 = 'https://financialmodelingprep.com/api/v3/company-key-metrics/' + tick + '?apikey=' + api_key
        url5 = 'https://financialmodelingprep.com/api/v3/financial-ratios/' + tick + '?apikey=' + api_key
        
        a0 = get_json_data(url0)
        a1 = get_json_data(url1)
        a2 = get_json_data(url2)
        a3 = get_json_data(url3)
        a4 = get_json_data(url4)
        a5 = get_json_data(url5)
    except:
        pass
    
    inc_statement_indicators.append(find_fin_indicators(a0,indicators))
    cash_flow_indicators.append(find_fin_indicators(a1,indicators))
    bs_indicators.append(find_fin_indicators(a2,indicators))
    fs_growth_indicators.append(find_fin_indicators(a3,indicators))
    metrics_indicators.append(find_fin_indicators(a4,indicators))
    ratios_indicators.append(find_ratios_indicators(a5,indicators))
    
    fin_indicators = inc_statement_indicators + cash_flow_indicators + bs_indicators + fs_growth_indicators + metrics_indicators + ratios_indicators
    fin_indicators_list = [item for sublist in fin_indicators for item in sublist]

100%|██████████| 2/2 [00:10<00:00,  5.49s/it]


In [88]:
# Transform the list of indicators in a DataFrame
temp = []
for j in fin_indicators_list:
    temp.append(j[1])
    df = pd.DataFrame(temp)
    df.columns = ['variable']
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
temp2 = []
for i in tickers_nyse:
    for j in fin_indicators_list:
        if j[0] == i:
            temp2.append(j[2])
            df_temp = pd.DataFrame(temp2)
            df_temp.columns = [j[0]]
            df_temp[j[0]] = df_temp[j[0]].astype(float)
    df = df.merge(df_temp, left_index = True, right_index = True)
    del df_temp
    temp2.clear()

In [89]:
# Set Symbol as Index in Stocks List
stocks_nyse_labelled = stocks_nyse.set_index('symbol')

In [90]:
# Set symbol as Index in Dataframe with Indicators
df_labelled = df.set_index('variable')
df_labelled_trasposed = df_labelled.T
df_labelled_trasposed = df_labelled_trasposed.reset_index()
data_final = df_labelled_trasposed.rename(columns={'index': 'symbol'})
data_final = data_final.set_index('symbol')
data_final = data_final.merge(stocks_nyse_labelled['sector'], left_on=['symbol'], right_on=['symbol'])

In [91]:
# Get 2020 returns and save results in a DataFrame
returns = []
for i in tickers_nyse:
    returns.append(get_price_var(i))
    
df_returns = pd.DataFrame(returns)
df_returns.columns = ['symbol','return']
df_returns = df_returns.set_index('symbol')

In [92]:
# Info on Final Data
data_final = data_final.merge(df_returns, left_on=['symbol'], right_on=['symbol'])
data_final.info();

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, AAPL to WMT
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Revenue Growth       2 non-null      float64
 1   EBIT Margin          2 non-null      float64
 2   Net Profit Margin    2 non-null      float64
 3   Investing Cash flow  2 non-null      float64
 4   Dividend payments    2 non-null      float64
 5   Total assets         2 non-null      float64
 6   Total liabilities    2 non-null      float64
 7   Net Income Growth    2 non-null      float64
 8   Debt Growth          2 non-null      float64
 9   Debt to Equity       2 non-null      float64
 10  Payout Ratio         2 non-null      float64
 11  returnOnEquity       2 non-null      float64
 12  assetTurnover        2 non-null      float64
 13  sector               2 non-null      object 
 14  return               2 non-null      float64
dtypes: float64(14), object(1)
memory usage: 256.